# LMDI Penetration Decomposition Demo (REST OF MARKET Approach)

This notebook demonstrates the LMDI penetration decomposition methodology with **rest-of-market competitor effects**.

## REST OF MARKET Approach

The key innovation is calculating competitor effects from **only competitors** (excluding the lender being analyzed):

- **Lender Effects**: Impact of the lender's own changes
- **Competitor Effects**: Impact of competitors' changes (no self-influence)
- **Net Effects**: Lender + Competitor for each driver

This provides **cleaner causal attribution** than using total market.

## Chart Types

| Chart | Description |
|-------|-------------|
| **Chart 1** | Original 8-Effect: 7 lender effects + 1 total competitor effect |
| **Chart 2** | Net Effects: 7 net effects (lender + competitor combined) |
| **Chart 3** | Lender vs Competitor: Stacked waterfall comparing contributions |
| **Chart 4** | Net Volume + Residual: Isolates volume as net, shows residual competitor |

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

# Import penetration decomposition calculator
from lmdi_penetration_calculator import (
    calculate_penetration_decomposition,
    calculate_multi_lender_penetration_decomposition
)

# Import penetration visualization functions
from visualization_penetration import (
    create_penetration_waterfall_grid,
    create_penetration_summary_table,
    print_penetration_decomposition
)

print("Modules loaded successfully!")

## Load Data

Load the mock funnel data with multiple lenders (needed to calculate market totals).

In [ ]:
# Load data
data_path = Path.cwd().parent / 'data' / 'funnel_data_2d_mock_with_nonfinanced.csv'
df = pd.read_csv(data_path)
df['month_begin_date'] = pd.to_datetime(df['month_begin_date'])

print(f"Data loaded: {len(df)} rows")
print(f"Date range: {df['month_begin_date'].min().date()} to {df['month_begin_date'].max().date()}")
print(f"Lenders: {df['lender'].unique().tolist()}")

## Calculate Penetration Decomposition

Calculate penetration decomposition for ACA between two periods.

### Methodology

**Penetration** = Lender_Bookings / Total_Market_Bookings

Using LMDI:
```
ΔP = Lender_Effect + Market_Effect

Lender_Effect = L(P₁,P₂) × ln(B_lender₂/B_lender₁)  → decomposed into 7 effects
Market_Effect = -L(P₁,P₂) × ln(B_market₂/B_market₁) → also decomposed into 7 effects

Net_Effect[i] = Lender_Effect[i] + Market_Effect[i]  (for each driver)
```

In [ ]:
# Define periods (1 year apart)
date_a = '2023-06-01'
date_b = '2024-06-01'
lender = 'ACA'

# Calculate penetration decomposition
results = calculate_penetration_decomposition(
    df=df,
    date_a=date_a,
    date_b=date_b,
    lender=lender
)

print(f"Penetration Decomposition for {lender}")
print(f"Period: {date_a} → {date_b}")

## Decomposition Summary

The new summary structure shows **lender, market, and net effects** for each driver.

In [ ]:
# Print formatted decomposition
print_penetration_decomposition(results.summary, results.metadata)

In [ ]:
# Display as table
summary_table = create_penetration_summary_table(results.summary, results.metadata, lender)
print("\nSummary Table:")
print(summary_table.to_string(index=False))

In [ ]:
# Display metadata
meta = results.metadata
print("=" * 60)
print("PENETRATION SUMMARY (SELF-ADJUSTED)")
print("=" * 60)
print(f"\nLender Bookings:")
print(f"  Period 1: {meta['period_1_lender_bookings']:,.0f}")
print(f"  Period 2: {meta['period_2_lender_bookings']:,.0f}")
print(f"  Change:   {meta['delta_lender_bookings']:+,.0f}")
print(f"\nTotal Market Bookings:")
print(f"  Period 1: {meta['period_1_total_market_bookings']:,.0f}")
print(f"  Period 2: {meta['period_2_total_market_bookings']:,.0f}")
print(f"  Change:   {meta['delta_total_market_bookings']:+,.0f}")
print(f"\nRest of Market (Competitors):")
print(f"  Period 1: {meta['period_1_rest_of_market_bookings']:,.0f}")
print(f"  Period 2: {meta['period_2_rest_of_market_bookings']:,.0f}")
print(f"  Change:   {meta['delta_rest_of_market_bookings']:+,.0f}")
print(f"\nPenetration:")
print(f"  Period 1: {meta['period_1_penetration']*100:.2f}%")
print(f"  Period 2: {meta['period_2_penetration']*100:.2f}%")
print(f"  Change:   {meta['delta_penetration_bps']:+.1f} bps")
print(f"\nEffect Totals:")
print(f"  Total Gross Lender Effect: {meta['total_gross_lender_effect_bps']:+.1f} bps")
print(f"  Total Self-Adjustment:     {meta['total_self_adjustment_bps']:+.1f} bps")
print(f"  Total Net Lender Effect:   {meta['total_net_lender_effect_bps']:+.1f} bps")
print(f"  Total Competitor Effect:   {meta['total_competitor_effect_bps']:+.1f} bps")
print(f"\nChart 4 Components:")
print(f"  Net Volume Effect:             {meta['net_volume_effect_bps']:+.1f} bps")
print(f"  Competitor Non-Volume Effect:  {meta['competitor_non_volume_effect_bps']:+.1f} bps")
print("=" * 60)

## Four Chart Views

The 2x2 grid shows four different perspectives on the penetration decomposition:

1. **Chart 1 (Top-Left)**: Original view - 7 lender effects + 1 total market effect
2. **Chart 2 (Top-Right)**: Net effects - 7 net effects (lender + market combined)
3. **Chart 3 (Bottom-Left)**: Lender vs Market - grouped bars comparing contributions
4. **Chart 4 (Bottom-Right)**: Net Volume + Residual - isolates volume as net

In [ ]:
# Create the 4-chart waterfall grid
fig = create_penetration_waterfall_grid(
    summary=results.summary,
    metadata=results.metadata,
    lender=lender,
    output_path='../outputs/penetration_waterfall_grid.png'
)

## Reconciliation Validation

Verify that all effects sum exactly to the actual penetration change across all views.

In [ ]:
# Validate reconciliation
actual_change = meta['delta_penetration_bps']

# Get effects (excluding total_change)
effects = results.summary[results.summary['effect_type'] != 'total_change']

# Check 1: Gross Lender effects sum
sum_gross_lender = effects['gross_lender_effect_bps'].sum()
gross_lender_error = abs(sum_gross_lender - meta['total_gross_lender_effect_bps'])

# Check 2: Net Lender effects sum
sum_net_lender = effects['net_lender_effect_bps'].sum()
net_lender_error = abs(sum_net_lender - meta['total_net_lender_effect_bps'])

# Check 3: Competitor effects sum
sum_competitor = effects['competitor_effect_bps'].sum()
competitor_error = abs(sum_competitor - meta['total_competitor_effect_bps'])

# Check 4: Net effects sum = actual change
sum_net = effects['net_effect_bps'].sum()
net_error = abs(sum_net - actual_change)

# Check 5: Chart 4 reconciliation
net_volume = meta['net_volume_effect_bps']
competitor_non_volume = meta['competitor_non_volume_effect_bps']
other_net_lender = effects[effects['effect_type'] != 'volume_effect']['net_lender_effect_bps'].sum()
chart4_sum = net_volume + other_net_lender + competitor_non_volume
chart4_error = abs(chart4_sum - actual_change)

print("=" * 60)
print("RECONCILIATION CHECKS (SELF-ADJUSTED)")
print("=" * 60)
print(f"\n1. Gross Lender Effects:")
print(f"   Sum of 7 effects: {sum_gross_lender:+.6f} bps")
print(f"   Total Gross:      {meta['total_gross_lender_effect_bps']:+.6f} bps")
print(f"   Error:            {gross_lender_error:.10f} bps")

print(f"\n2. Net Lender Effects:")
print(f"   Sum of 7 effects: {sum_net_lender:+.6f} bps")
print(f"   Total Net:        {meta['total_net_lender_effect_bps']:+.6f} bps")
print(f"   Error:            {net_lender_error:.10f} bps")

print(f"\n3. Competitor Effects:")
print(f"   Sum of 7 effects: {sum_competitor:+.6f} bps")
print(f"   Total Competitor: {meta['total_competitor_effect_bps']:+.6f} bps")
print(f"   Error:            {competitor_error:.10f} bps")

print(f"\n4. Net Effects (Chart 2):")
print(f"   Sum of 7 net:     {sum_net:+.6f} bps")
print(f"   Actual ΔP:        {actual_change:+.6f} bps")
print(f"   Error:            {net_error:.10f} bps")

print(f"\n5. Chart 4 Structure:")
print(f"   Net Volume:              {net_volume:+.6f} bps")
print(f"   Other Net Lender:        {other_net_lender:+.6f} bps")
print(f"   Competitor Non-Volume:   {competitor_non_volume:+.6f} bps")
print(f"   Sum:                     {chart4_sum:+.6f} bps")
print(f"   Actual ΔP:               {actual_change:+.6f} bps")
print(f"   Error:                   {chart4_error:.10f} bps")

print("\n" + "=" * 60)
all_passed = all([gross_lender_error < 0.01, net_lender_error < 0.01, 
                  competitor_error < 0.01, net_error < 0.01, chart4_error < 0.01])
if all_passed:
    print("✓ ALL RECONCILIATION CHECKS PASSED")
else:
    print("⚠ RECONCILIATION ERROR DETECTED")
print("=" * 60)

## Business Interpretation

Understand why penetration changed and attribute to specific drivers.

In [ ]:
# Business interpretation
print("=" * 70)
print("BUSINESS INTERPRETATION (SELF-ADJUSTED)")
print("=" * 70)

delta_pen = meta['delta_penetration_bps']
total_gross_lender = meta['total_gross_lender_effect_bps']
total_net_lender = meta['total_net_lender_effect_bps']
total_competitor = meta['total_competitor_effect_bps']

if delta_pen < 0:
    print(f"\n{lender}'s penetration DECREASED by {abs(delta_pen):.0f} bps")
else:
    print(f"\n{lender}'s penetration INCREASED by {delta_pen:.0f} bps")

print(f"\n  Gross Lender Effect: {total_gross_lender:+.0f} bps")
print(f"  Self-Adjustment:     {meta['total_self_adjustment_bps']:+.0f} bps")
print(f"  Net Lender Effect:   {total_net_lender:+.0f} bps")
if total_net_lender > 0:
    print(f"    → {lender}'s net performance would have INCREASED penetration")
else:
    print(f"    → {lender}'s net performance would have DECREASED penetration")

print(f"\n  Total Competitor Effect: {total_competitor:+.0f} bps")
if total_competitor < 0:
    print(f"    → Competitor growth REDUCED {lender}'s penetration")
else:
    print(f"    → Competitor contraction BOOSTED {lender}'s penetration")

print("\n" + "-" * 70)
print("Effect-by-Effect Analysis:")
print("-" * 70)

for _, row in effects.iterrows():
    effect_name = row['effect_type'].replace('_', ' ').title()
    gross_val = row['gross_lender_effect_bps']
    net_lender_val = row['net_lender_effect_bps']
    competitor_val = row['competitor_effect_bps']
    net_val = row['net_effect_bps']
    
    print(f"\n{effect_name}:")
    print(f"  Gross: {gross_val:+.0f} | Net Lender: {net_lender_val:+.0f} | Competitor: {competitor_val:+.0f} | Net: {net_val:+.0f} bps")
    
    if abs(net_val) > 20:  # Significant effect
        if net_val > 0:
            if net_lender_val > abs(competitor_val):
                print(f"  → Net positive: {lender}'s improvement outpaced competitors")
            else:
                print(f"  → Net positive: Competitor weakness helped {lender}")
        else:
            if abs(competitor_val) > net_lender_val:
                print(f"  → Net negative: Competitor growth exceeded {lender}'s improvement")
            else:
                print(f"  → Net negative: {lender}'s decline hurt penetration")

print("\n" + "=" * 70)

## Multi-Lender Penetration Analysis

Compare penetration decomposition across all lenders in the market.

In [ ]:
# Multi-lender penetration decomposition
multi_results = calculate_multi_lender_penetration_decomposition(
    df=df,
    date_a=date_a,
    date_b=date_b
)

In [ ]:
# Display multi-lender summary
print("\nMULTI-LENDER PENETRATION CHANGES (SELF-ADJUSTED)")
print("=" * 120)
print(multi_results.aggregate_summary.to_string(index=False))
print("=" * 120)
print("\nNote: Each lender's competitor effect excludes that lender (self-adjusted approach).")

In [ ]:
# Detailed comparison table
print("\nDetailed Effect Comparison by Lender (SELF-ADJUSTED):")
print("-" * 100)

for lender_name in multi_results.lender_details.keys():
    r = multi_results.lender_details[lender_name]
    print(f"\n{lender_name}:")
    effects_df = r.summary[r.summary['effect_type'] != 'total_change']
    for _, row in effects_df.iterrows():
        effect = row['effect_type'].replace('_', ' ').title()
        print(f"  {effect:<25} Gross:{row['gross_lender_effect_bps']:>+7.0f}  Net L:{row['net_lender_effect_bps']:>+7.0f}  Comp:{row['competitor_effect_bps']:>+7.0f}  Net:{row['net_effect_bps']:>+7.0f}")